In [1]:
from fpl_helpers import *
import pandas as pd

In [2]:
elements_df, teams_df = get_tables()
global PLAYER_IDS
PLAYER_IDS = elements_df[['full_name', 'id']].set_index('full_name').to_dict()['id']
PLAYER_NAMES = {PLAYER_IDS[i]:i for i in list(PLAYER_IDS.keys()) }


In [3]:
def get_team(team_id, gw):
  url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/"
  r = requests.get(url)
  json = r.json()
  return json

team_ids = [i['element'] for i in get_team(715692, 10)['picks']]
team_names = [PLAYER_NAMES[i] for i in team_ids]

In [4]:
team_names

['David Raya Martin',
 'Pau Torres',
 'Rayan Aït-Nouri',
 'Gabriel dos Santos Magalhães',
 'Leif Davis',
 'Morgan Rogers',
 'Mohamed Salah',
 'Cole Palmer',
 'Liam Delap',
 'Chris Wood',
 'Erling Haaland',
 'Hákon Valdimarsson',
 'Taylor Harwood-Bellis',
 'Amadou Onana',
 'Yankuba Minteh']

In [5]:
compare_players(team_names[5:11], metrics=ATTACKER_METRICS, agg='mean')

,Difficulty,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected
Morgan Rogers,"4.0, 2.0, 2.0, 2.0, 5.0, 4.0",4.00,0.33,0.37,0.00,0.03,0.41,0.67,12.33,16.93,22.90,20.67,6.07,1.00,2384038.67
Mohamed Salah,"5.0, 4.0, 2.0, 2.0, 4.0, 2.0",11.67,1.33,0.92,0.33,0.19,1.10,1.67,39.33,32.13,16.47,32.33,8.10,1.00,5110231.67
Cole Palmer,"3.0, 2.0, 4.0, 2.0, 2.0, 2.0",2.33,0.00,0.42,0.00,0.15,0.58,0.33,15.67,9.87,28.43,18.67,5.70,1.00,5867021.67
Liam Delap,"3.0, 2.0, 2.0, 2.0, 2.0, 3.0",4.33,0.33,0.33,0.33,0.02,0.35,1.00,16.00,16.40,11.20,13.00,4.07,0.67,536910.67
Chris Wood,"2.0, 5.0, 3.0, 3.0, 3.0, 4.0",3.33,0.33,0.26,0.00,0.01,0.27,0.33,15.33,16.20,10.57,14.67,4.13,0.67,2371165.00
Erling Haaland,"5.0, 3.0, 2.0, 3.0, 4.0, 2.0",3.00,0.33,1.07,0.00,0.02,1.09,0.00,9.00,14.40,8.00,73.33,9.60,1.00,7113521.33


In [6]:
forwards = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==4)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
forwards['difficulty'] = forwards['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
forwards['avg_difficulty'] = forwards['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
forwards['cost_per_point'] = forwards['now_cost']/forwards['total_points']
#forwards[:20].sort_values(by='difficulty')
forwards

,full_name,team,selected_by_percent,now_cost,value_season,total_points,expected_goals,goals_scored,expected_assists,assists,expected_goals_conceded,goals_conceded,difficulty,avg_difficulty,cost_per_point
423,Erling Haaland,Man City,59.4,151,5.6,84,11.92,12,0.46,0,15.83,17,"[5.0, 3.0, 2.0, 3.0, 4.0, 2.0]",3.166667,1.797619
653,Matheus Santos Carneiro Da Cunha,Wolves,13.7,69,10.9,75,3.07,7,2.29,3,18.51,25,"[2.0, 2.0, 2.0, 2.0, 2.0, 3.0]",2.166667,0.920000
209,Nicolas Jackson,Chelsea,18.5,79,9.1,72,6.39,7,0.53,4,13.73,12,"[3.0, 2.0, 4.0, 2.0, 2.0, 2.0]",2.500000,1.097222
533,Chris Wood,Nott'm Forest,23.4,66,10.5,69,4.45,8,0.19,0,9.62,10,"[2.0, 5.0, 3.0, 3.0, 3.0, 4.0]",3.333333,0.956522
170,Danny Welbeck,Brighton,21.9,60,11.0,66,5.11,6,0.37,2,18.72,13,"[2.0, 3.0, 2.0, 2.0, 2.0, 2.0]",2.166667,0.909091
63,Ollie Watkins,Aston Villa,22.5,90,6.9,62,6.43,6,0.59,2,11.40,14,"[4.0, 2.0, 2.0, 2.0, 5.0, 4.0]",3.166667,1.451613
126,Yoane Wissa,Brentford,8.4,61,9.2,56,5.30,6,0.34,2,11.06,11,"[2.0, 4.0, 3.0, 4.0, 3.0, 3.0]",3.166667,1.089286
323,Liam Delap,Ipswich,4.8,56,9.1,51,2.51,6,0.13,1,17.75,17,"[3.0, 2.0, 2.0, 2.0, 2.0, 3.0]",2.333333,1.098039
677,Jørgen Strand Larsen,Wolves,3.0,56,8.9,50,2.61,4,0.48,2,17.72,24,"[2.0, 2.0, 2.0, 2.0, 2.0, 3.0]",2.166667,1.120000
302,Raúl Jiménez,Fulham,9.4,57,8.8,50,4.31,4,1.31,2,9.37,10,"[4.0, 2.0, 5.0, 5.0, 2.0, 4.0]",3.666667,1.140000


In [15]:
forward_scores = compare_players(forwards['full_name'], metrics=ATTACKER_METRICS, agg='mean')
forward_scores.data['Average Difficulty'] = forward_scores.data['Difficulty'].apply(lambda x: np.mean(x))
#forward_scores

In [8]:
preferred_low = ["Average Difficulty"]

In [10]:
weights = {
    'Average Difficulty':-2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':0,
    'Expected Assists':0, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':10
}
df = calculate_weighted_score(forward_scores.data.drop('Difficulty', axis=1), weights, preferred_low)
df

,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Matheus Santos Carneiro Da Cunha,11.33,1.00,0.29,1.00,0.23,0.52,2.33,47.00,56.67,27.70,29.67,11.40,1.00,942878.00,2.17,17.80
Dominic Solanke-Mitchell,7.67,0.67,0.67,0.67,0.02,0.70,1.00,31.67,37.07,7.80,43.00,8.80,1.00,1473901.33,3.00,15.61
Francisco Evanilson de Lima Barbosa,5.67,0.67,0.67,0.00,0.01,0.68,1.00,23.00,27.67,8.83,38.33,7.47,1.00,137968.00,2.50,15.58
Erling Haaland,3.00,0.33,1.07,0.00,0.02,1.09,0.00,9.00,14.40,8.00,73.33,9.60,1.00,7113521.33,3.17,14.24
Yoane Wissa,5.67,0.67,0.48,0.00,0.01,0.49,1.00,22.67,23.13,7.10,20.00,4.97,1.00,727061.00,3.17,14.13
Jørgen Strand Larsen,5.00,0.33,0.26,0.33,0.02,0.28,0.67,21.00,21.33,13.17,20.33,5.47,1.00,306459.33,2.17,13.40
Adam Armstrong,5.00,0.67,0.42,0.33,0.07,0.49,0.33,22.00,12.73,8.73,12.67,3.43,1.00,62775.67,3.33,13.26
Nicolas Jackson,5.00,0.33,0.19,0.33,0.09,0.28,1.00,24.00,22.13,13.43,36.33,7.20,1.00,1874667.33,2.50,13.15
Ollie Watkins,3.67,0.33,0.70,0.00,0.01,0.71,0.33,7.67,13.87,1.57,31.00,4.30,1.00,2604584.33,3.17,13.13
Darwin Núñez Ribeiro,3.67,0.33,0.41,0.00,0.02,0.43,0.33,18.00,13.73,5.40,26.00,4.53,1.00,257699.00,3.17,12.36


In [11]:
midfielders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==3)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
midfielders['difficulty'] = midfielders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
midfielders['avg_difficulty'] = midfielders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
midfielders['cost_per_point'] = midfielders['now_cost']/midfielders['total_points']
#forwards[:20].sort_values(by='difficulty')
#midfielders

In [12]:
mid_weights = {
    'Average Difficulty': -2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':1,
    'Expected Assists':1, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':1
}

midfielder_scores = compare_players(midfielders['full_name'], metrics=ATTACKER_METRICS, agg='mean')
midfielder_scores.data['Average Difficulty'] = midfielder_scores.data['Difficulty'].apply(lambda x: np.mean(x))
df = calculate_weighted_score(midfielder_scores.data.drop('Difficulty', axis=1), mid_weights, preferred_low)
df


,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Mohamed Salah,11.33,1.33,0.95,0.33,0.19,1.13,1.67,38.00,32.13,16.47,32.33,8.10,1.00,5110231.67,3.17,11.08
Bruno Borges Fernandes,9.00,0.67,0.50,0.67,0.09,0.59,2.00,33.33,41.73,43.53,15.67,10.07,0.67,1198409.33,3.17,7.74
Brennan Johnson,5.33,0.67,0.83,0.00,0.02,0.85,0.33,17.67,24.27,2.77,34.33,5.90,0.67,1096213.67,3.00,5.50
Bukayo Saka,5.67,0.33,0.17,0.33,0.17,0.34,1.00,21.00,22.93,25.87,30.67,7.93,1.00,2714142.33,2.33,5.01
Antoine Semenyo,4.33,0.33,0.37,0.00,0.13,0.50,1.00,23.33,22.13,21.33,42.33,8.50,1.00,976030.00,2.50,4.93
James Maddison,7.67,1.00,0.30,0.00,0.02,0.31,1.00,29.67,40.60,1.87,19.00,6.13,0.33,705751.33,3.00,4.85
Mitoma Kaoru,4.33,0.33,0.25,0.00,0.17,0.42,0.67,16.67,18.27,16.90,23.00,5.83,1.00,579264.00,2.17,4.36
Son Heung-min,4.00,0.00,0.05,0.67,0.43,0.48,0.00,21.00,21.93,32.40,19.00,7.37,1.00,695141.00,3.00,3.63
Cole Palmer,2.33,0.00,0.42,0.00,0.15,0.58,0.33,15.67,9.87,28.43,18.67,5.70,1.00,5867021.67,2.50,3.57
Dejan Kulusevski,4.33,0.00,0.04,0.67,0.39,0.43,0.00,20.33,25.87,37.57,13.00,7.63,1.00,419006.00,3.00,3.48


In [13]:
defenders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==2)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
defenders['difficulty'] = defenders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
defenders['avg_difficulty'] = defenders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
defenders['cost_per_point'] = defenders['now_cost']/defenders['total_points']
defender_scores = compare_players(defenders['full_name'], metrics=DEFENDER_METRICS, agg='mean')
defender_scores.data['Average Difficulty'] = defender_scores.data['Difficulty'].apply(lambda x: np.mean(x))


In [14]:
def_weights ={
 'Average Difficulty': -2,
 'Total Points': 3,
 'Clean Sheets': 1,
 'Goals Conceded': -1,
 'Own Goals': -1,
 'Yellow Cards': -1,
 'Bonus': 1.5,
 'Bps': 1,
 'Influence': 1,
 'Creativity': 1,
 'Threat': 1,
 'Ict Index': 1,
 'Starts': 1,
 'Expected Goal Involvements': 1.5,
 'Expected Goals Conceded': -1.5
 }

df = calculate_weighted_score(defender_scores.data.drop('Difficulty', axis=1), def_weights)
df

,Total Points,Minutes,Clean Sheets,Goals Conceded,Own Goals,Yellow Cards,Red Cards,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Expected Goal Involvements,Expected Goals Conceded,Average Difficulty,total_score
James Tarkowski,5.33,90.00,0.67,0.33,0.00,0.33,0.00,1.00,25.33,28.93,14.93,11.67,5.57,1.00,0.22,0.88,4.00,7.31
Pedro Porro,6.33,90.00,0.33,1.00,0.00,0.33,0.00,0.67,21.00,34.07,23.20,9.67,6.70,1.00,0.18,1.92,3.00,6.96
Ashley Young,5.33,90.00,0.67,0.33,0.00,0.33,0.00,1.00,22.00,20.07,38.43,2.00,6.07,1.00,0.13,0.88,4.00,6.71
Milos Kerkez,4.00,86.67,0.00,2.00,0.00,0.00,0.00,0.67,19.33,28.93,24.43,8.00,6.13,1.00,0.25,1.32,2.50,6.25
William Saliba,3.67,90.00,0.33,0.67,0.00,0.00,0.00,0.33,23.00,19.80,9.27,2.00,3.13,1.00,0.17,0.74,2.33,5.70
Antonee Robinson,4.00,90.00,0.33,1.67,0.00,0.00,0.00,0.33,17.67,25.80,27.60,12.33,6.60,1.00,0.25,1.12,3.67,5.49
Leif Davis,3.67,60.00,0.00,0.67,0.00,0.33,0.00,0.67,10.00,23.67,23.23,8.67,5.57,0.67,0.17,1.12,2.33,5.07
Murillo Santiago Costa dos Santos,4.67,90.00,0.33,2.00,0.00,0.00,0.00,0.00,16.00,36.07,7.93,13.33,5.73,1.00,0.13,1.08,3.33,4.80
Ola Aina,5.33,89.33,0.33,2.00,0.00,0.33,0.00,1.00,14.00,23.40,6.63,5.67,3.57,1.00,0.06,1.08,3.33,4.62
Gabriel dos Santos Magalhães,3.33,90.00,0.33,0.67,0.00,0.00,0.00,0.00,15.67,16.87,9.70,3.33,3.00,1.00,0.14,0.74,2.33,4.58
